Section 0 — Pre-requisites (paths + imports)

In [1]:
import os, sys
from pathlib import Path
import pandas as pd

# --- Robust project root detection (works even if CWD is weird) ---
NOTEBOOK_DIR = Path.cwd().resolve()

root = NOTEBOOK_DIR
for _ in range(10):
    if (root / "src").exists() and (root / "data").exists():
        break
    root = root.parent
else:
    raise RuntimeError("Could not find project root (folder containing 'src' and 'data').")

PROJECT_ROOT = root
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
OUTPUTS_DIR = PROJECT_ROOT / "outputs"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)
print("src exists:", (PROJECT_ROOT / "src").exists())
print("data exists:", (PROJECT_ROOT / "data").exists())

XWALK_PATH = RAW_DIR / "county_cbsa_crosswalk.xls"
print("XWALK_PATH:", XWALK_PATH)
print("XWALK exists:", XWALK_PATH.exists())

API_KEY = os.getenv("CENSUS_API_KEY")
print("CENSUS_API_KEY set:", bool(API_KEY))


NOTEBOOK_DIR: /Users/lilsebo/Downloads/flo_capex_model/notebooks
PROJECT_ROOT: /Users/lilsebo/Downloads/flo_capex_model
src exists: True
data exists: True
XWALK_PATH: /Users/lilsebo/Downloads/flo_capex_model/data/raw/county_cbsa_crosswalk.xls
XWALK exists: True
CENSUS_API_KEY set: True


Section 1 — Load config (candidates + Flo hubs + weights)

In [2]:
from src.config import CANDIDATE_METROS, FLO_HUBS, BASE_WEIGHTS

print("Candidate metros:", len(CANDIDATE_METROS))
print("FLO_HUBS:", FLO_HUBS)
print("BASE_WEIGHTS:", BASE_WEIGHTS)

pd.DataFrame({"candidate": CANDIDATE_METROS})


Candidate metros: 14
FLO_HUBS: ['Phoenix, AZ', 'Dallas-Fort Worth, TX', 'San Diego, CA']
BASE_WEIGHTS: {'demand': 0.3, 'enterprise_fit': 0.2, 'cloud_adjacency': 0.2, 'resilience': 0.15, 'capex_friction': -0.15}


,candidate
0,"San Diego, CA"
1,"Los Angeles, CA"
2,"Inland Empire, CA"
3,"Phoenix, AZ"
4,"Tucson, AZ"
5,"El Paso, TX"
6,"Laredo, TX"
7,"McAllen, TX"
8,"Brownsville, TX"
9,"San Antonio, TX"


Section 2 — Load crosswalk (county → CBSA)

In [3]:
from src.ingest import load_county_cbsa_crosswalk

xwalk = load_county_cbsa_crosswalk(XWALK_PATH)

print("Crosswalk rows:", len(xwalk))
print("Crosswalk columns:", list(xwalk.columns))
xwalk.head()


Crosswalk rows: 1866
Crosswalk columns: ['cbsa_code', 'cbsa_name', 'fips_county']


,cbsa_code,cbsa_name,fips_county
0,10020,"Abbeville, LA",22113
1,10100,"Aberdeen, SD",46013
2,10100,"Aberdeen, SD",46045
3,10140,"Aberdeen, WA",53027
4,10180,"Abilene, TX",48059


Section 3 — Load CBP county data (safe: uses cache if API key missing)

In [4]:
from src.ingest import fetch_cbp_county_by_naics

YEAR = 2022
NAICS_CODES = ["31-33", "48-49", "51", "52", "54"]

CACHE_PATH = PROCESSED_DIR / f"cbp_county_{YEAR}_selected_naics.csv"
print("CACHE_PATH:", CACHE_PATH)
print("Cache exists:", CACHE_PATH.exists())

if (not API_KEY) and CACHE_PATH.exists():
    # Your ingest.fetch_cbp_county_by_naics() raises if API key missing,
    # so we bypass it and load cache directly.
    cbp = pd.read_csv(CACHE_PATH, dtype=str)
    print("Loaded CBP from cache (no API key needed). Rows:", len(cbp))
else:
    cbp = fetch_cbp_county_by_naics(
        year=YEAR,
        naics_list=NAICS_CODES,
        api_key=API_KEY,
        cache_path=CACHE_PATH,
        force_refresh=False
    )
    print("Fetched CBP via API. Rows:", len(cbp))

print("CBP columns:", list(cbp.columns))
cbp.head()


CACHE_PATH: /Users/lilsebo/Downloads/flo_capex_model/data/processed/cbp_county_2022_selected_naics.csv
Cache exists: True
Fetched CBP via API. Rows: 14915
CBP columns: ['EMP', 'ESTAB', 'NAICS2017', 'STATE', 'COUNTY', 'NAICS2017.1', 'state', 'county']


,EMP,ESTAB,NAICS2017,STATE,COUNTY,NAICS2017.1,state,county
0,1281,28,31-33,01,001,31-33,01,001
1,4071,181,31-33,01,003,31-33,01,003
2,2107,21,31-33,01,005,31-33,01,005
3,4082,91,31-33,01,055,31-33,01,055
4,172,17,31-33,04,007,31-33,04,007


Section 4 — Create county FIPS + normalize expected fields

In [5]:
# Expected columns from your ingest.py: EMP, ESTAB, NAICS2017, STATE, COUNTY (but cache may load as strings)
need_cols = ["EMP", "ESTAB", "NAICS2017", "STATE", "COUNTY"]
missing = [c for c in need_cols if c not in cbp.columns]
if missing:
    raise KeyError(f"CBP data missing columns {missing}. Has: {list(cbp.columns)}")

cbp["fips_county"] = cbp["STATE"].astype(str).str.zfill(2) + cbp["COUNTY"].astype(str).str.zfill(3)

# Force numeric EMP/ESTAB
cbp["EMP"] = pd.to_numeric(cbp["EMP"], errors="coerce").fillna(0).astype(int)
cbp["ESTAB"] = pd.to_numeric(cbp["ESTAB"], errors="coerce").fillna(0).astype(int)

cbp[["fips_county", "NAICS2017", "EMP", "ESTAB"]].head()


,fips_county,NAICS2017,EMP,ESTAB
0,01001,31-33,1281,28
1,01003,31-33,4071,181
2,01005,31-33,2107,21
3,01055,31-33,4082,91
4,04007,31-33,172,17


Section 5 — Merge CBP with crosswalk (attach cbsa_code/cbsa_name)

In [6]:
need_x = ["fips_county", "cbsa_code", "cbsa_name"]
missing_x = [c for c in need_x if c not in xwalk.columns]
if missing_x:
    raise KeyError(f"Crosswalk missing columns {missing_x}. Has: {list(xwalk.columns)}")

cbp = cbp.merge(xwalk[need_x], on="fips_county", how="left")

print("After merge rows:", len(cbp))
cbp[["fips_county", "cbsa_code", "cbsa_name", "NAICS2017", "EMP"]].head()


After merge rows: 14915


,fips_county,cbsa_code,cbsa_name,NAICS2017,EMP
0,01001,33860,"Montgomery, AL",31-33,1281
1,01003,19300,"Daphne-Fairhope-Foley, AL",31-33,4071
2,01005,21640,"Eufaula, AL-GA",31-33,2107
3,01055,23460,"Gadsden, AL",31-33,4082
4,04007,37740,"Payson, AZ",31-33,172


Section 6 — Keep only valid CBSA rows + enforce cbsa_code as int

In [7]:
cbp = cbp.dropna(subset=["cbsa_code"]).copy()
cbp["cbsa_code"] = pd.to_numeric(cbp["cbsa_code"], errors="coerce")
cbp = cbp.dropna(subset=["cbsa_code"]).copy()
cbp["cbsa_code"] = cbp["cbsa_code"].astype(int)

print("Rows with valid cbsa_code:", len(cbp))
cbp[["cbsa_code", "cbsa_name", "NAICS2017", "EMP", "ESTAB"]].head()


Rows with valid cbsa_code: 8988


,cbsa_code,cbsa_name,NAICS2017,EMP,ESTAB
0,33860,"Montgomery, AL",31-33,1281,28
1,19300,"Daphne-Fairhope-Foley, AL",31-33,4071,181
2,21640,"Eufaula, AL-GA",31-33,2107,21
3,23460,"Gadsden, AL",31-33,4082,91
4,37740,"Payson, AZ",31-33,172,17


Section 7 — Aggregate counties → metros → NAICS

In [8]:
metro_naics = (
    cbp.groupby(["cbsa_code", "cbsa_name", "NAICS2017"], as_index=False)[["EMP", "ESTAB"]]
       .sum()
)

print("Metro-NAICS rows:", len(metro_naics))
metro_naics.head()


Metro-NAICS rows: 4735


,cbsa_code,cbsa_name,NAICS2017,EMP,ESTAB
0,10020,"Abbeville, LA",31-33,491,36
1,10020,"Abbeville, LA",48-49,454,49
2,10020,"Abbeville, LA",51,117,14
3,10020,"Abbeville, LA",52,322,64
4,10020,"Abbeville, LA",54,295,108


Section 8 — Pivot NAICS buckets into sector columns

In [9]:
naics_bucket = {
    "31-33": "manufacturing",
    "48-49": "logistics",
    "51": "information",
    "52": "finance",
    "54": "professional_services",
}

metro_naics["industry"] = metro_naics["NAICS2017"].astype(str).map(naics_bucket)

metro_features = (
    metro_naics.dropna(subset=["industry"])
              .pivot_table(
                  index=["cbsa_code", "cbsa_name"],
                  columns="industry",
                  values="EMP",
                  aggfunc="sum",
                  fill_value=0
              )
              .reset_index()
)

metro_features.columns.name = None
print("Metro features rows:", len(metro_features))
metro_features.head()


Metro features rows: 949


,cbsa_code,cbsa_name,finance,information,logistics,manufacturing,professional_services
0,10020,"Abbeville, LA",322,117,454,491,295
1,10100,"Aberdeen, SD",978,135,429,3642,460
2,10140,"Aberdeen, WA",483,130,432,2050,399
3,10180,"Abilene, TX",3106,1015,2836,3632,1887
4,10220,"Ada, OK",498,87,297,1427,883


Section 9 — Demand index

In [10]:
# Ensure all needed columns exist
for col in ["manufacturing", "logistics", "professional_services", "finance", "information"]:
    if col not in metro_features.columns:
        metro_features[col] = 0

metro_features["demand_index"] = (
    0.40 * metro_features["manufacturing"] +
    0.30 * metro_features["logistics"] +
    0.20 * metro_features["professional_services"] +
    0.10 * metro_features["finance"]
)

metro_features.sort_values("demand_index", ascending=False).head(10)


,cbsa_code,cbsa_name,finance,information,logistics,manufacturing,professional_services,demand_index
616,35620,"New York-Northern New Jersey-Long Island, NY-N...",548831,347947,358394,295829,773191,435371.1
506,31100,"Los Angeles-Long Beach-Santa Ana, CA",258711,258653,234186,442058,457711,364492.3
165,16980,"Chicago-Joliet-Naperville, IL-IN-WI",274845,98209,267322,376158,396282,337400.7
216,19100,"Dallas-Fort Worth-Arlington, TX",253949,101692,239546,277698,290653,266468.5
51,12060,"Atlanta-Sandy Springs-Marietta, GA",129779,105783,178898,162084,257635,183007.9
909,47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV",111232,109093,68384,50213,632717,178267.0
392,26420,"Houston-Sugar Land-Baytown, TX",97303,42633,140676,199467,223889,176497.7
674,37980,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",184391,70269,121225,171911,229428,169456.6
108,14460,"Boston-Cambridge-Quincy, MA-NH",154149,121099,75791,167982,318079,168960.8
234,19820,"Detroit-Warren-Livonia, MI",99182,33541,72285,232525,210003,166614.3


Section 10 — Parse reporting fields (primary city + state)

In [11]:
def parse_cbsa_primary_city(cbsa_name: str) -> str:
    s = str(cbsa_name)
    left = s.split(",")[0]
    return left.split("-")[0].strip()

def parse_cbsa_state(cbsa_name: str) -> str:
    s = str(cbsa_name)
    parts = s.split(",")
    return parts[1].strip() if len(parts) > 1 else ""

metro_features["primary_city"] = metro_features["cbsa_name"].apply(parse_cbsa_primary_city)
metro_features["state"] = metro_features["cbsa_name"].apply(parse_cbsa_state)

metro_features.head()


,cbsa_code,cbsa_name,finance,information,logistics,manufacturing,professional_services,demand_index,primary_city,state
0,10020,"Abbeville, LA",322,117,454,491,295,423.8,Abbeville,LA
1,10100,"Aberdeen, SD",978,135,429,3642,460,1775.3,Aberdeen,SD
2,10140,"Aberdeen, WA",483,130,432,2050,399,1077.7,Aberdeen,WA
3,10180,"Abilene, TX",3106,1015,2836,3632,1887,2991.6,Abilene,TX
4,10220,"Ada, OK",498,87,297,1427,883,886.3,Ada,OK


Section 11 — Robust candidate mapping (short names → CBSA code) + tag is_candidate

In [12]:
def _norm(s: str) -> str:
    return str(s).strip().upper()

# Normalize CBSA names for matching
metro_features["cbsa_name_u"] = metro_features["cbsa_name"].astype(str).map(_norm)

def map_candidate_to_cbsa(candidate: str) -> dict:
    cand_u = _norm(candidate)

    # Parse "City, ST" if possible
    if "," in cand_u:
        city = cand_u.split(",")[0].strip()
        st = cand_u.split(",")[1].strip()
    else:
        city, st = cand_u, ""

    # Special case: "Inland Empire, CA" ≈ Riverside-San Bernardino-Ontario, CA CBSA
    if "INLAND EMPIRE" in city and st == "CA":
        hits = metro_features[
            metro_features["cbsa_name_u"].str.contains("RIVERSIDE", na=False) &
            metro_features["cbsa_name_u"].str.contains("SAN BERNARDINO", na=False)
        ]
        if len(hits) > 0:
            best = hits.sort_values("demand_index", ascending=False).iloc[0]
            return {
                "candidate_input": candidate,
                "matched": True,
                "match_method": "special_case_inland_empire",
                "cbsa_code": int(best["cbsa_code"]),
                "cbsa_name": best["cbsa_name"],
            }

    # General case: require state abbreviation to appear in CBSA state list (right side of comma)
    # and city to be either a prefix (starts with) or contained.
    hits = metro_features.copy()
    if st:
        hits = hits[hits["cbsa_name_u"].str.contains(st, na=False)]

    # City heuristic
    hits = hits[
        hits["cbsa_name_u"].str.startswith(city) |
        hits["cbsa_name_u"].str.contains(city, na=False)
    ]

    if len(hits) == 0:
        return {
            "candidate_input": candidate,
            "matched": False,
            "match_method": "no_hit",
            "cbsa_code": None,
            "cbsa_name": None,
        }

    # Tie-break: pick the highest demand_index among matches
    best = hits.sort_values("demand_index", ascending=False).iloc[0]
    return {
        "candidate_input": candidate,
        "matched": True,
        "match_method": "heuristic_city_state",
        "cbsa_code": int(best["cbsa_code"]),
        "cbsa_name": best["cbsa_name"],
    }

# Build mapping table
mapped_rows = [map_candidate_to_cbsa(c) for c in CANDIDATE_METROS]
cand_map = pd.DataFrame(mapped_rows)

print("Matched candidates:", int(cand_map["matched"].sum()), "/", len(cand_map))
display(cand_map)

# Tag metros by CBSA code (robust)
matched_codes = set(cand_map.loc[cand_map["matched"] == True, "cbsa_code"].astype(int).tolist())
metro_features["is_candidate"] = metro_features["cbsa_code"].isin(matched_codes)

# Save mapping for your executive summary / appendix
cand_map_path = OUTPUTS_DIR / "candidate_cbsa_mapping.csv"
cand_map.to_csv(cand_map_path, index=False)
print("Saved candidate mapping:", cand_map_path)

# Report unmatched (e.g., Monterrey, MX)
unmatched = cand_map.loc[cand_map["matched"] == False, "candidate_input"].tolist()
if unmatched:
    print("\nUnmatched candidates (expected if not a US CBSA):")
    for u in unmatched:
        print(" -", u)

# Quick view: candidates that are tagged in metro universe
metro_features[metro_features["is_candidate"]].sort_values("demand_index", ascending=False).head(20)[
    ["cbsa_code","cbsa_name","state","demand_index","manufacturing","logistics","information","finance","professional_services"]
]


Matched candidates: 13 / 14


,candidate_input,matched,match_method,cbsa_code,cbsa_name
0,"San Diego, CA",True,heuristic_city_state,41740.0,"San Diego-Carlsbad-San Marcos, CA"
1,"Los Angeles, CA",True,heuristic_city_state,31100.0,"Los Angeles-Long Beach-Santa Ana, CA"
2,"Inland Empire, CA",True,special_case_inland_empire,40140.0,"Riverside-San Bernardino-Ontario, CA"
3,"Phoenix, AZ",True,heuristic_city_state,38060.0,"Phoenix-Mesa-Glendale, AZ"
4,"Tucson, AZ",True,heuristic_city_state,46060.0,"Tucson, AZ"
5,"El Paso, TX",True,heuristic_city_state,21340.0,"El Paso, TX"
6,"Laredo, TX",True,heuristic_city_state,29700.0,"Laredo, TX"
7,"McAllen, TX",True,heuristic_city_state,32580.0,"McAllen-Edinburg-Mission, TX"
8,"Brownsville, TX",True,heuristic_city_state,15180.0,"Brownsville-Harlingen, TX"
9,"San Antonio, TX",True,heuristic_city_state,41700.0,"San Antonio-New Braunfels, TX"


Saved candidate mapping: /Users/lilsebo/Downloads/flo_capex_model/outputs/candidate_cbsa_mapping.csv

Unmatched candidates (expected if not a US CBSA):
 - Monterrey, MX


,cbsa_code,cbsa_name,state,demand_index,manufacturing,logistics,information,finance,professional_services
506,31100,"Los Angeles-Long Beach-Santa Ana, CA",CA,364492.3,442058,234186,258653,258711,457711
216,19100,"Dallas-Fort Worth-Arlington, TX",TX,266468.5,277698,239546,101692,253949,290653
392,26420,"Houston-Sugar Land-Baytown, TX",TX,176497.7,199467,140676,42633,97303,223889
676,38060,"Phoenix-Mesa-Glendale, AZ",AZ,130437.0,134187,104450,41448,151432,151420
725,40140,"Riverside-San Bernardino-Ontario, CA",CA,100165.7,103900,158594,15865,27063,41606
762,41740,"San Diego-Carlsbad-San Marcos, CA",CA,92770.6,100034,35792,36081,58138,181028
59,12420,"Austin-Round Rock-San Marcos, TX",TX,58931.6,49717,26680,55037,47076,131666
761,41700,"San Antonio-New Braunfels, TX",TX,52905.2,51922,43129,18028,74531,58723
868,46060,"Tucson, AZ",AZ,17616.1,22016,12004,6176,13633,19226
270,21340,"El Paso, TX",TX,13775.7,15645,16430,6531,7885,9001


Section 12 — Save processed dataset (FULL universe)

In [13]:
OUT_PATH = PROCESSED_DIR / "market_features.csv"
metro_features.to_csv(OUT_PATH, index=False)

print("Saved:", OUT_PATH)
print("Rows saved:", len(metro_features))
print("Candidate-tagged rows:", int(metro_features["is_candidate"].sum()))


Saved: /Users/lilsebo/Downloads/flo_capex_model/data/processed/market_features.csv
Rows saved: 949
Candidate-tagged rows: 13


Section 13 — QA checks (top overall + top candidates)

In [14]:
from IPython.display import display

print("Total metros:", len(metro_features))
print("Columns:", list(metro_features.columns))

print("\nTop 10 overall by demand_index")
display(
    metro_features.sort_values("demand_index", ascending=False)
    .loc[:, ["cbsa_code","cbsa_name","primary_city","state","demand_index","is_candidate"]]
    .head(10)
)

print("\nTop candidates by demand_index")
display(
    metro_features[metro_features["is_candidate"]]
    .sort_values("demand_index", ascending=False)
    .loc[:, ["cbsa_code","cbsa_name","primary_city","state","demand_index"]]
    .head(20)
)


Total metros: 949
Columns: ['cbsa_code', 'cbsa_name', 'finance', 'information', 'logistics', 'manufacturing', 'professional_services', 'demand_index', 'primary_city', 'state', 'cbsa_name_u', 'is_candidate']

Top 10 overall by demand_index


,cbsa_code,cbsa_name,primary_city,state,demand_index,is_candidate
616,35620,"New York-Northern New Jersey-Long Island, NY-N...",New York,NY-NJ-PA,435371.1,False
506,31100,"Los Angeles-Long Beach-Santa Ana, CA",Los Angeles,CA,364492.3,True
165,16980,"Chicago-Joliet-Naperville, IL-IN-WI",Chicago,IL-IN-WI,337400.7,False
216,19100,"Dallas-Fort Worth-Arlington, TX",Dallas,TX,266468.5,True
51,12060,"Atlanta-Sandy Springs-Marietta, GA",Atlanta,GA,183007.9,False
909,47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Washington,DC-VA-MD-WV,178267.0,False
392,26420,"Houston-Sugar Land-Baytown, TX",Houston,TX,176497.7,True
674,37980,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Philadelphia,PA-NJ-DE-MD,169456.6,False
108,14460,"Boston-Cambridge-Quincy, MA-NH",Boston,MA-NH,168960.8,False
234,19820,"Detroit-Warren-Livonia, MI",Detroit,MI,166614.3,False



Top candidates by demand_index


,cbsa_code,cbsa_name,primary_city,state,demand_index
506,31100,"Los Angeles-Long Beach-Santa Ana, CA",Los Angeles,CA,364492.3
216,19100,"Dallas-Fort Worth-Arlington, TX",Dallas,TX,266468.5
392,26420,"Houston-Sugar Land-Baytown, TX",Houston,TX,176497.7
676,38060,"Phoenix-Mesa-Glendale, AZ",Phoenix,AZ,130437.0
725,40140,"Riverside-San Bernardino-Ontario, CA",Riverside,CA,100165.7
762,41740,"San Diego-Carlsbad-San Marcos, CA",San Diego,CA,92770.6
59,12420,"Austin-Round Rock-San Marcos, TX",Austin,TX,58931.6
761,41700,"San Antonio-New Braunfels, TX",San Antonio,TX,52905.2
868,46060,"Tucson, AZ",Tucson,AZ,17616.1
270,21340,"El Paso, TX",El Paso,TX,13775.7
